### Exp: 2 Cluster customers first... Then map each cluster to vehicle

Assumption: Customer comes in cluster big small medium with some noise(somebody crossing da road) clustering them might mean they comes from the same vehicle (or come together from somewhere)

Cross : Create a HDBSCAN cluster then tune it accordingly

Cross OUTPUT : 
- df_cross['cluster'] : the cluster that the that customer belongs to with 
- cluster_cross : the cluster info of cross containin 
    - xy mid mean, min , max - timestamp (start stop avg) and COUNT of cluster size


camera used : mbk-14-12-vehicle 

### Load

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

pd.set_option('display.precision', 10)
CURRENT_CAM = "mbk-14-12"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)
# print(df_vehicle.to_string())
# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT]].to_string())

### Preprocess

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

### Clustering the customer

In [4]:
df_cross ,  cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, 'cluster']].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:56: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-co

In [5]:
print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST].to_string())

           timestamp_min        timestamp_max  count  cluster_id        xmid_mean       ymid_mean
0    2024-04-28 02:00:08  2024-04-28 15:31:18     61          -1  1670.8770491803  323.5000000000
1    2024-04-28 02:11:31  2024-04-28 02:12:48      2           0  1336.7500000000  430.0000000000
7    2024-04-28 02:23:46  2024-04-28 02:25:27      5           6  1004.9000000000  495.8000000000
8    2024-04-28 02:27:18  2024-04-28 02:27:24      3           7  2107.6666666667  105.3333333333
3    2024-04-28 02:34:14  2024-04-28 02:34:21      3           2  2133.1666666667   58.6666666667
2    2024-04-28 02:44:29  2024-04-28 02:48:43      3           1  1876.3333333333  261.8333333333
9    2024-04-28 03:03:14  2024-04-28 03:03:36      8           8  1127.8125000000  693.0625000000
10   2024-04-28 03:03:48  2024-04-28 03:04:09      6           9  1241.1666666667  617.1666666667
45   2024-04-28 03:04:33  2024-04-28 03:04:47      4          44  1147.8750000000  728.5000000000
46   2024-04-28 03:0

result: The clustering now seems good enough with time_biased_distance4 metric (care more on time than coordinate and time diff is more than x amount it probably is not from the same group)

In [6]:
john =  Cluster.assign_vehicle_to_cluster(df_vehicle, cluster_cross)
# print(john[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT_CLUSTERED]].to_string())
# sucks ass

TRY assigning vehicle to cluster first ( then เก็บตก large cluster only?)

In [7]:
tony = Cluster.assign_cross_cluster_to_vehicle_in_lifetime(df_vehicle,cluster_cross,distance_metric="euclidean")
print(tony[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT_CLUSTERED]].to_string())

                 camera       timestamp_precise         lifetime    xmid   ymid plate_number vehicle_type  timestamp_unix  timestamp_unix_end  count          cluster_list
2315  mbk-14-12-vehicle 2024-04-28 02:27:41.480    14.2359998226  1706.0  194.0       327722       pickup      1714271261          1714271275      0                    []
2242  mbk-14-12-vehicle 2024-04-28 03:08:09.313    18.3550000191  1564.5  181.5      4กบ5887       pickup      1714273689          1714273707      0                    []
2241  mbk-14-12-vehicle 2024-04-28 03:08:27.068    49.8659999371  1397.0  140.5      6กถ9907       pickup      1714273707          1714273756      0                    []
2215  mbk-14-12-vehicle 2024-04-28 03:18:58.899    46.8269999027  1551.5  172.0          อก5       pickup      1714274338          1714274384      0                    []
2204  mbk-14-12-vehicle 2024-04-28 03:21:31.980     2.6389999390  1401.5  174.0       ทห1197       pickup      1714274491          1714274493    


bad case : 06:47:56.476   516.1789999008  1747.0  200.0       340223       pickup      1714286876          1714287392      9        [87, 111, 151, 152]

TODO: find a better metric function to calculate distances